In [1]:
import numpy as np
import pandas as pd
import torch
# from processAllData import getAllReadyForStationByLatAndLongAndK, getAllReadyForStationByLatAndLongAndKSplitTestAndTrain

#from LSTMArchitecture import GHIDataset, Main_LSTM
import torch.nn as nn
import glob # For loading multiple files
#import random
import os

from constants import COLUMN_NAMES
%load_ext autoreload
%autoreload 2

In [47]:
def cyclicalEncoding(data, cycleLength):
  newDatasin = np.sin(2 * np.pi * (data - 1) / cycleLength)  # Sine encoding for hours (adjust for 0-23)
  newDatacos = np.cos(2 * np.pi * (data - 1) / cycleLength)  # Cosine encoding for hours (adjust for 0-23)
  return newDatasin, newDatacos

# YEAR_MONTH_DAY_HOUR = COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]
usecols = [
    'Year Month Day Hour (YYYYMMDDHH)',
    'Opaque sky cover / 0-10 in tenths',
    'Global horizontal irradiance / kJ/m2',
    'Direct normal irradiance / kJ/m2',
    'Diffuse horizontal irradiance / kJ/m2',
    'Wind direction / 0-359 degrees',
    'Wind speed / 0.1 m/s',
    'Dry bulb temperature / 0.1 C',
]

dtype = {
    'Year Month Day Hour (YYYYMMDDHH)': str,  # Read as string initially to handle errors
    'Opaque sky cover / 0-10 in tenths': str,
    'Global horizontal irradiance / kJ/m2': str,
    'Direct normal irradiance / kJ/m2': str,
    'Diffuse horizontal irradiance / kJ/m2': str,
    'Wind direction / 0-359 degrees': str,
    'Wind speed / 0.1 m/s': str,
    'Dry bulb temperature / 0.1 C': str,
}

def fix_opaque_sky_cover(df):
  #If opaque sky cover is nan then set to most recent opaque sky cover value as long as its not been more than 3 hours since last value
  last_valid_value = None
  last_valid_index = -1
  num_skipped = 0
  min_year = df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]].min()
  max_year = df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]].max()
  #print(f"Before fixing opaque sky cover, min year: {min_year}, max year: {max_year}")
  for i in range(len(df)):
    if df.at[i, COLUMN_NAMES["OPAQUE_SKY_COVER"]] != 99 and not np.isnan(df.at[i, COLUMN_NAMES["OPAQUE_SKY_COVER"]]):
      last_valid_value = df.at[i, COLUMN_NAMES["OPAQUE_SKY_COVER"]]
      last_valid_index = i
      num_skipped = 0
    else:
      num_skipped += 1
      if last_valid_value is not None and num_skipped < 8:
        df.at[i, COLUMN_NAMES["OPAQUE_SKY_COVER"]] = last_valid_value
    if num_skipped >= 8:
      # More than 8 consecutive invalid entries, stop filling and remove the rest of the rows in the dataframe.
      if last_valid_index == -1:
        # No valid entries found, return empty dataframe
        return df.iloc[0:0], False, False
      df = df.iloc[:last_valid_index+5]  # Keep up to 4 hours after the last valid entry
      max_year = df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]].max()
      break
  if np.isnan(max_year):
    max_year = min_year
  
  return df, min_year, max_year

def get_stations_data(station_csv):
  # Read the CSV file with specified columns and data types
  df = pd.read_csv(station_csv, delimiter=',', skiprows=2, index_col=False, usecols=usecols, dtype=dtype, on_bad_lines='skip')
  # Convert columns to numeric, coercing errors to NaN
  station_name_df = pd.read_csv(station_csv, delimiter=',', nrows=1, usecols=['Climate station name'], on_bad_lines='skip')
  station_name = station_name_df.iloc[0, 0]
  for col in usecols:
    if(col == COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]):
      #keep as string
      df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]] = df[col].astype(int)
      df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]] = df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]].astype(str)
      # Get year (YYYY)
      df[COLUMN_NAMES["YEAR"]] = df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]].str[0:4].astype(int)
      # Get month (1-12)
      df[COLUMN_NAMES["MONTH"]] = df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]].str[4:6].astype(int)
      # Get hour (1-24)
      df[COLUMN_NAMES["HOUR"]] = df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]].str[8:10].astype(int)
      df[COLUMN_NAMES["HOUR_SIN"]], df[COLUMN_NAMES["HOUR_COS"]] = cyclicalEncoding(df[COLUMN_NAMES["HOUR"]], 24)# Sine and Cosine encoding for hours (adjust for 0-23)
      # df['Hour_sin'] = np.sin(2 * np.pi * (df['Hour'] - 1) / 24)
      # df['Hour_cos'] = np.cos(2 * np.pi * (df['Hour'] - 1) / 24)  # Cosine encoding for hours (adjust for 0-23)
      # Normalize month so cyclic (0-11))
      df[COLUMN_NAMES["MONTH_SIN"]], df[COLUMN_NAMES["MONTH_COS"]] = cyclicalEncoding(df[COLUMN_NAMES["MONTH"]], 12)# Sine and Cosine encoding for months
      df[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]] = df[col].astype(int)
      #df = df.drop(columns=[col], axis=1)
      continue

    if (col == 'Opaque sky cover / 0-10 in tenths'):
      df[col] = pd.to_numeric(df[col], errors='coerce')
      #Skip rest here and handle later for df_cloud
      continue

    if(col=='Wind direction / 0-359 degrees'):
      df[col] = pd.to_numeric(df[col], errors='coerce').replace(99, np.nan)
      radians = np.deg2rad(df[col])
      df[COLUMN_NAMES["WIND_DIRECTION_SIN"]] = np.sin(radians)
      df[COLUMN_NAMES["WIND_DIRECTION_COS"]] = np.cos(radians)
      #df = df.drop(columns=[col], axis=1)
      continue

    if(col == 'Dry bulb temperature / 0.1 C'):
      df[col] = pd.to_numeric(df[col], errors='coerce').replace(999, np.nan)
      # mean = df[col].mean()
      # std = df[col].std()
      # df[col] = (df[col] - mean) / std
      continue

    if(col == 'Wind speed / 0.1 m/s'):
      df[col] = pd.to_numeric(df[col], errors='coerce').replace(99, np.nan)
      # mean = df[col].mean()
      # std = df[col].std()
      # df[col] = (df[col] - mean) / std
      continue

    if (col == 'Global horizontal irradiance / kJ/m2' or col == 'Direct normal irradiance / kJ/m2' or col == 'Diffuse horizontal irradiance / kJ/m2'):
      df[col] = pd.to_numeric(df[col], errors='coerce').replace(9999, np.nan)
      # mean = df[col].mean()
      # std = df[col].std()
      #df[col] = (df[col] - mean) / std #can denormalize output
      continue
    
    df[col] = pd.to_numeric(df[col], errors='coerce')
  df.fillna(df.mean(numeric_only=True), inplace=True) #Replace NaN with mean of column for all columns except opaque sky cover which is kept as 99 instead of NaN 
  df_renamed = df.rename(columns={
      'Global horizontal irradiance / kJ/m2': COLUMN_NAMES["GHI"],
      'Direct normal irradiance / kJ/m2': COLUMN_NAMES["DNI"],
      'Diffuse horizontal irradiance / kJ/m2': COLUMN_NAMES["DHI"],
      'Wind speed / 0.1 m/s': COLUMN_NAMES["WIND_SPEED"],
      'Opaque sky cover / 0-10 in tenths': COLUMN_NAMES["OPAQUE_SKY_COVER"],
      'Wind direction / 0-359 degrees': COLUMN_NAMES["WIND_DIRECTION"],
      "Dry bulb temperature / 0.1 C": COLUMN_NAMES["DRY_BULB_TEMPERATURE"]
  })
  df_cloud = df_renamed.copy()
  df_cloud, min_year, max_year = fix_opaque_sky_cover(df_cloud)
  #print(f"After fixing opaque sky cover for {station_csv}, min year: {min_year}, max year: {max_year}")
  #min_year = int(min_year)
  #max_year = int(max_year)

  df_renamed = df_renamed.drop(columns=[COLUMN_NAMES["OPAQUE_SKY_COVER"]], axis=1)
  #df_cloud = df_cloud[df_cloud[YEAR_MONTH_DAY_HOUR]>=min_year]
  #df_cloud = df_cloud[df_cloud[YEAR_MONTH_DAY_HOUR]<=max_year]
  #df_cloud = df_cloud.drop(columns=[YEAR_MONTH_DAY_HOUR], axis=1)
  #df_renamed = df_renamed.drop(columns=[YEAR_MONTH_DAY_HOUR], axis=1)
  has_cloud_data = True
  if not min_year:
    has_cloud_data = False
    
  return df_cloud, df_renamed, station_name.replace(" ", "_"), has_cloud_data

def chunk(df, interval = 25):
  results = []
  for i in range(0, len(df)-interval, 1):
    chunk = df[i:i+interval]
    results.append(chunk)
  results = np.array(results)
  return results

def get_chunked_tensors(nearest_stations, dfs, interval):
  chunked_tensors = []
  rows = 0
  station_order = []
  for index, station in nearest_stations.iterrows():
    distanceVector = station.iloc[3][1]
    chunked_df = chunk(dfs[rows], interval=interval)
    rows+=1
    chunkedTensor = torch.tensor(chunked_df).to(torch.float32)
    chunked_tensors.append(chunkedTensor)
    station_order.append(station)
  return chunked_tensors, station_order


def create_cleaned_csv_files(csv_files, save_path_cloud, save_path_non_cloud):
  for f in csv_files:
    df_cloud, df_no_cloud, station_name, has_cloud_data = get_stations_data(f)
    print(f"Saving cleaned data for station {station_name} to: ")
    # Save the cleaned DataFrames to the specified paths
    
    save_path_non_cloud_station = save_path_non_cloud + '/' + station_name + '_no_sky_cover.csv'
    print(f"{save_path_non_cloud_station}")
    if has_cloud_data: #Only saving if data exists
      save_path_cloud_station = save_path_cloud + '/' + station_name + '_sky_cover.csv'
      df_cloud.to_csv(save_path_cloud_station, index=False)
      print(f"and {save_path_cloud_station} ")
    
    df_no_cloud.to_csv(save_path_non_cloud_station, index=False)


In [48]:
path = 'Datasets/CWEEDS_2020_BC_raw'
csv_files = glob.glob(os.path.join(path, "*.csv"))
save_path_cloud = 'Datasets/CWEEDS_2020_BC_cleaned_cloud'
save_path_non_cloud = 'Datasets/CWEEDS_2020_BC_cleaned_non_cloud'
create_cleaned_csv_files(csv_files, save_path_cloud, save_path_non_cloud)

Saving cleaned data for station ABBOTSFORD_A to: 
Datasets/CWEEDS_2020_BC_cleaned_non_cloud/ABBOTSFORD_A_no_sky_cover.csv
and Datasets/CWEEDS_2020_BC_cleaned_cloud/ABBOTSFORD_A_sky_cover.csv 
Saving cleaned data for station AGASSIZ_RCS to: 
Datasets/CWEEDS_2020_BC_cleaned_non_cloud/AGASSIZ_RCS_no_sky_cover.csv
Saving cleaned data for station BALLENAS_ISLAND to: 
Datasets/CWEEDS_2020_BC_cleaned_non_cloud/BALLENAS_ISLAND_no_sky_cover.csv
Saving cleaned data for station BLUE_RIVER_CS to: 
Datasets/CWEEDS_2020_BC_cleaned_non_cloud/BLUE_RIVER_CS_no_sky_cover.csv
Saving cleaned data for station BONILLA_ISLAND_(AUT) to: 
Datasets/CWEEDS_2020_BC_cleaned_non_cloud/BONILLA_ISLAND_(AUT)_no_sky_cover.csv
Saving cleaned data for station BURNS_LAKE_DECKER_LAKE to: 
Datasets/CWEEDS_2020_BC_cleaned_non_cloud/BURNS_LAKE_DECKER_LAKE_no_sky_cover.csv
Saving cleaned data for station CALLAGHAN_VALLEY to: 
Datasets/CWEEDS_2020_BC_cleaned_non_cloud/CALLAGHAN_VALLEY_no_sky_cover.csv
Saving cleaned data for st